In this notebook we will process the files inside the folder DatasetsInUse

First, we will start to process the data from the emotion_tweets_2020 folder

In [192]:
import re
import inflect
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import pandas as pd

def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.read().splitlines()
    return lines

def mapping(key_list):
    emotion_map = {
        '0': "anger",
        '1': "joy",
        '2': "optimism",
        '3': "sadness",
    }

    emotions = [emotion_map[num] for num in key_list]

    return emotions

def specific_case(text):
    result = re.sub(r'(&gt;){3}', 'is better than', text)
    result = result.replace("szn", "season")
    result = re.sub(r'&[gl]t;?', '', result)
    result = result.replace("ó", "o")
    result = result.replace("ñ", "n")
    result = result.replace("é", "e")
    return result

def normalize_repeated_characters(text):
    # Replace 3 or more consecutive characters with just one
    return re.sub(r'(.)\1{2,}', r'\1', text)

def remove_user_mentions(text):
    return re.sub(r'@(\w+)', r'\1', text)

def process_more_sign(text):
    result = re.sub(r'\s*user \+', 'user', text)
    result = re.sub(r'#\++', '', result)
    result = re.sub(r'(?<=\d)\+', ' more ', result)
    result = re.sub(r'(?<=\s)\+', ' plus ', result)
    result = re.sub(r'\+1', ' plus one ', result)
    return result

def process_dollar(text):
    result = re.sub(r'\${2,}', 'cash', text)
    pattern = r'\$(\d+(?:\.\d{2})?)'
    result = re.sub(pattern, lambda match: match.group(1) + ' dollars ', result)
    result = re.sub(r'\$*', '', result)
    return result

def process_euro(text):
    pattern = r'\€(\d+(?:\.\d{2})?)'
    result = re.sub(pattern, lambda match: match.group(1) + ' euros ', text)
    return result

def process_pounds(text):
    pattern = r'\£(\d+(?:\.\d{2})?)'
    result = re.sub(pattern, lambda match: match.group(1) + ' pounds ', text)
    return result

def process_percent(text):
    pattern = r'(?:\s+|\d+(?:\.\d{0,2})?)%'
    result = re.sub(pattern, lambda match: match.group(0).replace('%', ' percent '), text)
    result = re.sub(r'%', '', result)
    return result

def process_equal(text):
    result = re.sub(r'=', ' equals ', text)
    return result

def process_at(text):
    result = re.sub(r'(?<=\s)@(?=\s)', ' at ', text)
    return result

def remove_newlines(text):
    return re.sub(r'\\n', ' ', text)

def process_amp(text):
    return re.sub(r'&amp;?', ' and ', text)

def process_hyphen(text):
    return re.sub(r'(\d+)\s*-\s*(\d+)', r'\1 to \2', text)

def replace_numbers_with_words(text):
    p = inflect.engine()

    number_pattern = r'(\d+\.\d+|\d+)'

    numbers = re.findall(number_pattern, text)

    for number in numbers:
        word_representation = p.number_to_words(number)
        text = re.sub(re.escape(number), word_representation, text)

    return text

def clear_special_characters(text):
    return re.sub(r'[^a-zA-Z0-9\s]', ' ', text)

def lowercase_text(text):
    return text.lower()

def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    tokens = nltk.word_tokenize(text)
    return ' '.join(lemmatizer.lemmatize(word, wordnet.VERB) for word in tokens)

def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))
    tokens = nltk.word_tokenize(text)
    return ' '.join(word for word in tokens if word.lower() not in stop_words)

def process_text(text):
    text = specific_case(text)
    text = remove_user_mentions(text)
    text = process_more_sign(text)
    text = process_dollar(text)
    text = process_euro(text)
    text = process_pounds(text)
    text = process_percent(text)
    text = process_hyphen(text)
    text = process_equal(text)
    text = process_at(text)
    text = remove_newlines(text)
    text = process_amp(text)
    text = replace_numbers_with_words(text)
    text = normalize_repeated_characters(text)
    text = clear_special_characters(text)
    text = lowercase_text(text)
    text = lemmatize_text(text)
    text = remove_stopwords(text)
    return text

def create_df(keys_file, values_file):
    keys = read_file(keys_file)
    keys = mapping(keys)    
    values = read_file(values_file)

    processed_values = [process_text(value) for value in values]

    #for value, processed_value in zip(values, processed_values):
        #print(f"Original Value: {value} - Processed Value: {processed_value}")

    # Create a dictionary to store the data
    data_dict = {
        "text": processed_values,
        "emotions": keys  # Use the processed values
    }

    # Create a Pandas DataFrame from the dictionary
    df = pd.DataFrame(data_dict)

    # Display the DataFrame
    # df.head()

    return df

def create_dictionary(keys_file, values_file):
    keys = read_file(keys_file)
    keys = mapping(keys)    
    values = read_file(values_file)
    
    # print(keys[:5])

    # print(values[2839])

    if len(keys) != len(values):
        print("Error: The number of keys and values does not match.")
        return None

    non_alphanumeric_characters = {}
    
    for key, value in zip(keys, values):
        
        #if key == "Optimism":
        #    print(value)
        # Use a regular expression to find non-alphanumeric characters
        #ats = []
        #ats = re.findall(r"(\d)\1{2,}", value)
        characters = re.findall(r'[^a-zA-Z0-9\s]', value)
        for character in characters:
            if character not in non_alphanumeric_characters:
                non_alphanumeric_characters[character] = value

        #if ats != []:
            #print(value)

    print("Non-alphanumeric characters in the text:")
    for ch, sent in non_alphanumeric_characters.items():
        print(ch)
        print(sent)

    return emotion_tweets

keys_file = "DatasetsInUse/emotion_tweets_2020/train_labels.txt"
values_file = "DatasetsInUse/emotion_tweets_2020/train_text.txt"

df = create_df(keys_file, values_file)

print(df.shape[0])

df.head()

# resulting_dict = create_dictionary(keys_file, values_file)

#for key in list(resulting_dict.keys())[:3]:
    #value = resulting_dict[key]
    #print(key, value)

3257


,text,emotions
0,worry payment problem may never joyce meyer mo...,optimism
1,roommate okay spell autocorrect terrible first...,anger
2,cute atsu probably shy photos cherry help uwu,joy
3,rooneys fuck untouchable fuck dreadful depay l...,anger
4,pretty depress u hit pan ur favourite highlighter,sadness


Secondly, we will process the data from the emotion folder

In [5]:
import pandas as pd
import re

df = pd.read_pickle("DatasetsInUse/emotion/merged_training.pkl")

non_alphanumeric_characters = {}

for row in df.itertuples():
    characters = re.findall(r'[^a-zA-Z0-9\s]', row.text)
    for character in characters:
            if character not in non_alphanumeric_characters:
                non_alphanumeric_characters[character] = row.text
    #print(row.emotions, row.text)

# print("Non-alphanumeric characters in the text:")
for ch, sent in non_alphanumeric_characters.items():
    print(ch)
    print(sent)

print(df.emotions.unique())
#df.head()
#print(df.shape[0])

['sadness' 'joy' 'love' 'anger' 'fear' 'surprise']


In [45]:
import re

def normalize_repeated_characters(text):
    # Replace 3 or more consecutive characters with just one
    return re.sub(r'(.)\1{2,}', r'\1', text)

# Example
text = "I'm sooooo happyyyy today!!!"
normalized_text = normalize_repeated_characters(text)

print(normalized_text)

I'm so happy today!


In [53]:
def remove_user_mentions(text):
    return re.sub(r'@\w+', 'user', text)

# Example
text = "Hey @user, great post!"
cleaned_text = remove_user_mentions(text)

print(cleaned_text)

Hey user, great post!


In [65]:
import string

def remove_special_characters(text):
    return ''.join(char for char in text if char not in string.punctuation)

# Example
text = "Hello, Twitter! £§£€5 #NLP"
cleaned_text = remove_special_characters(text)
cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', ' ', cleaned_text)

print(cleaned_text)

Hello Twitter     5 NLP


In [185]:
def replace_szn_with_season(text):
    # Use the str.replace() method to replace "szn" with "season"
    #result = text.replace("szn", "season")
    result = text.replace("é", "e")
    return result

# Example
original_text = "@user szn 3 &gt;&gt;&gt; szn 1 &gt;&gt;&gt; szn 2. Just to warn you. Don't let szn 2 discourage you. "
text = "bdjdb berbéu é jhdbé éhdhdh ggdéff"
modified_text = replace_szn_with_season(text)

print("Original Text:", text)
print("Modified Text:", modified_text)

Original Text: bdjdb berbéu é jhdbé éhdhdh ggdéff
Modified Text: bdjdb berbeu e jhdbe ehdhdh ggdeff


In [100]:
import re

def replace_is_better_than(text):
    # Use re.sub to replace "is better than" with ">>>"
    result = re.sub(r'(&gt;){3}', 'is better than', text)
    return result

# Example
original_text = "@user szn 3 &gt;&gt;&gt; szn 1 &gt;&gt;&gt; szn 2. Just to warn you. Don't let szn 2 discourage you. "
modified_text = replace_is_better_than(original_text)

print("Original Text:", original_text)
print("Modified Text:", modified_text)

Original Text: @user szn 3 &gt;&gt;&gt; szn 1 &gt;&gt;&gt; szn 2. Just to warn you. Don't let szn 2 discourage you. 
Modified Text: @user szn 3 is better than szn 1 is better than szn 2. Just to warn you. Don't let szn 2 discourage you. 


In [108]:
import re

def replace_user_plus_at_beginning(text):
    pattern = r'\s*user \+'
    replacement = 'user'
    result = re.sub(pattern, replacement, text)
    return result

# Examples
text1 = ' user + account created'
text2 = 'user + account created'
text3 = '    user + account created'

print(replace_user_plus_at_beginning(text1))  # 'user account created'
print(replace_user_plus_at_beginning(text2))  # 'user+ account created'
print(replace_user_plus_at_beginning(text3))  # 'user account created'


user account created
user account created
user account created


In [112]:
import re

def find_more_than_one_dollar_signs(text):
    pattern = r'\${2,}'
    matches = re.findall(pattern, text)
    return matches

# Example
text = "The price is $$10.00, $$$25.50, and $$$$50.75."
result = find_more_than_one_dollar_signs(text)
print(result)  # ['$$', '$$$', '$$$$']

['$$', '$$$', '$$$$']


In [152]:
import re

def add_dollars_to_numbers(text):
    # Define a regular expression pattern to match a dollar sign followed by a number
    pattern = r'\$(\d+(?:\.\d{2})?)'
    
    # Use re.sub with a lambda function to replace matched text
    result = re.sub(pattern, lambda match: match.group(1) + ' dollars', text)
    
    return result

# Example
text = "The price is $10.00, but the total is $50.75."
result = add_dollars_to_numbers(text)
print(result)  # "The price is 10.00 dollars, but the total is 50.75 dollars."


The price is 10.00 dollars, but the total is 50.75 dollars.


In [119]:
import re

def replace_percentages_with_string(text):
    pattern = r'(?:\s+|\d+(?:\.\d{0,2})?)%'
    
    # Use re.sub with a lambda function to replace matched text
    result = re.sub(pattern, lambda match: match.group(0).replace('%', ' percent '), text)
    
    return result

# Example
text = "The interest rate is 5%, and the discount is 10.25 %."
result = replace_percentages_with_string(text)
print(result)  # "The interest rate is 5 percent, and the discount is 10.25 percent."


The interest rate is 5 percent , and the discount is 10.25  percent .


In [124]:
import re

text = "Send me an email at user@example.com. Mention me @ mention if you have questions."

matches = re.findall(r'\s@\s', text)

print(matches)


[' @ ']


In [132]:
import re

text = "Send me an email at user@example.com. Mention me @mention if you have questions."

# Use a capturing group to capture the words after "@" and replace the entire match
result = re.sub(r'@(\w+)', r'\1', text)

print(result)


Send me an email at userexample.com. Mention me mention if you have questions.


In [151]:
import re

def process_at(text):
    result = re.sub(r'(?<=\s)@(?=\s)', ' at ', text)
    # result = re.sub(r'(?:\s)(@)(?:\s)', lambda match: match.group(1).replace('@', ' at '), text)
    return result

# Example usage
text = "Send me an email at user @ example.com. Mention me @ mention if you have questions."

result = process_at(text)
print(result)


Send me an email at user  at  example.com. Mention me  at  mention if you have questions.


In [158]:
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    tokens = nltk.word_tokenize(text)
    return ' '.join(lemmatizer.lemmatize(word, wordnet.VERB) for word in tokens)

# Example
text = "I am                running in        the           park               ."
lemmatized_text = lemmatize_text(text)

print(lemmatized_text)

I be run in the park .


In [179]:
import pandas as pd

# Sample data
keys = ["Anger", "Joy", "Optimism", "Sadness", "Joy", "Joy"]
values = ["I am Rachel", "You are out of your mind", "Lalalala", "No more", "Yay", "Let's go!"]

# Define a function to process each element in the "Values" list
def process_value(value):
    # Apply your processing function here
    return value.upper()  # For example, convert to uppercase

# Apply the processing function to each element in the "Values" list
processed_values = [process_value(value) for value in values]

# Create a dictionary to store the data
data_dict = {
    "text": processed_values,
    "emotions": keys  # Use the processed values
}

# Create a Pandas DataFrame from the dictionary
df = pd.DataFrame(data_dict)

# Display the DataFrame
df.head()


,text,emotions
0,I AM RACHEL,Anger
1,YOU ARE OUT OF YOUR MIND,Joy
2,LALALALA,Optimism
3,NO MORE,Sadness
4,YAY,Joy


In [24]:
import pandas as pd
import json

column_names = ['Text', 'Emotion', 'ID']

# Replace 'your_file.tsv' with the path to your TSV file
df = pd.read_csv('DatasetsInUse/go_emotion/train.tsv', delimiter='\t', names=column_names)

# function to divide each emotion string into a list of numbers
# after that, the numbers are mapped to the corresponding emotion in emotions.txt
# then these emotions are grouped

with open('DatasetsInUse/go_emotion/emotions.txt', 'r') as file:
    # Read the contents of the file
    file_contents = file.read()

lines = file_contents.split('\n')

print(lines)

map = {index: line for index, line in enumerate(lines)}

print(map)

# Specify the path to your local JSON file
file_path = 'DatasetsInUse/go_emotion/ekman_mapping.json'

try:
    # Open the JSON file for reading
    with open(file_path, 'r') as json_file:
        # Load and parse the JSON data
        ekman_map = json.load(json_file)

except FileNotFoundError:
    print(f'JSON file not found: {file_path}')
except Exception as e:
    print(f'An error occurred: {str(e)}')

print(ekman_map)

# Create a reverse lookup dictionary
ekman_map = {tuple(value): key for key, value in ekman_map.items()}

print(ekman_map)

# ekman mapping of the values

ekman_list = [value for emotion in lines for keys, value in ekman_map.items() if emotion in keys]

print(ekman_list)

def emotion_mapping(emotions, emotion_map, ekman_map):
    numbers_list = emotions.split(',')
    emotions_list = [emotion_map[num] for num in numbers_list]
    # group the emotions
    ekman_list = [value for emotion in emotions_list for keys, value in ekman_map.items() if emotion in keys]
    duplicates = [(item, ekman_list.count(item)) for item in set(ekman_list) if ekman_list.count(item) > 1]
    sorted_duplicates = sorted(duplicates, key=lambda x: x[1], reverse=True)
    if sorted_duplicates == []:
        return ekman_list[0]
    else:
        return sorted_duplicates[0][0]

# Now, 'df' contains your data in a DataFrame
#print(df['Emotion'].unique())

['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']
{0: 'admiration', 1: 'amusement', 2: 'anger', 3: 'annoyance', 4: 'approval', 5: 'caring', 6: 'confusion', 7: 'curiosity', 8: 'desire', 9: 'disappointment', 10: 'disapproval', 11: 'disgust', 12: 'embarrassment', 13: 'excitement', 14: 'fear', 15: 'gratitude', 16: 'grief', 17: 'joy', 18: 'love', 19: 'nervousness', 20: 'optimism', 21: 'pride', 22: 'realization', 23: 'relief', 24: 'remorse', 25: 'sadness', 26: 'surprise', 27: 'neutral'}
{'anger': ['anger', 'annoyance', 'disapproval'], 'disgust': ['disgust'], 'fear': ['fear', 'nervousness'], 'joy': ['joy', 'amusement', 'approval', 'excitement', 'gratitude', 'love', 'optimism', 'relief', 'pride', 'admiration', 'des

In [4]:
# Replace 'your_file.txt' with the path to your file
file_path = 'DatasetsInUse/go_emotion/train.tsv'

# Initialize a line count variable
line_count = 0

# Open the file and count lines
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        line_count += 1

# Print the total number of lines
print("Total number of lines:", line_count)


Total number of lines: 43410


In [35]:
my_list = [4, 2, 4, 2, 4, 4, 2, 4]

# Find duplicates and create a list of tuples with element and count
duplicates = [(item, my_list.count(item)) for item in set(my_list) if my_list.count(item) > 1]

sorted_duplicates = sorted(duplicates, key=lambda x: x[1], reverse=True)

print(sorted_duplicates[0][0])

print(sorted_duplicates)


4
[(4, 5), (2, 3)]


In [135]:
import pandas as pd
import json

column_names = ['Text', 'Emotion', 'ID']

df = pd.read_csv('DatasetsInUse/go_emotion/train.tsv', delimiter='\t', names=column_names)

with open('DatasetsInUse/go_emotion/emotions.txt', 'r') as file:
    # Read the contents of the file
    file_contents = file.read()

lines = file_contents.split('\n')

map = {index: line for index, line in enumerate(lines)}

file_path = 'DatasetsInUse/go_emotion/ekman_mapping.json'

try:
    # Open the JSON file for reading
    with open(file_path, 'r') as json_file:
        # Load and parse the JSON data
        ekman_map = json.load(json_file)

except FileNotFoundError:
    print(f'JSON file not found: {file_path}')
except Exception as e:
    print(f'An error occurred: {str(e)}')

# Create a reverse lookup dictionary
ekman_map = {tuple(value): key for key, value in ekman_map.items()}

# function to divide each emotion string into a list of numbers
# after that, the numbers are mapped to the corresponding emotion in emotions.txt
# then these emotions are grouped
def emotion_mapping(emotions, emotion_map, ekman_map):
    numbers_list = emotions.split(',')
    #print(numbers_list)
    emotions_list = [emotion_map[int(num)] for num in numbers_list]
    #print(emotions_list)
    # group the emotions
    ekman_list = [value for emotion in emotions_list for keys, value in ekman_map.items() if emotion in keys]
    duplicates = [(item, ekman_list.count(item)) for item in set(ekman_list) if ekman_list.count(item) > 1]
    sorted_duplicates = sorted(duplicates, key=lambda x: x[1], reverse=True)
    #print(ekman_list)
    #print(sorted_duplicates)
    if sorted_duplicates == []:
        if ekman_list == []:
            return 'neutral'
        else:
            return ekman_list[0]
    else:
        return sorted_duplicates[0][0]

df['Emotion'] = df['Emotion'].apply(emotion_mapping, args = (map, ekman_map))
df.shape[0]
df.Emotion.unique()

array(['neutral', 'anger', 'fear', 'surprise', 'joy', 'sadness',
       'disgust'], dtype=object)

In [125]:
import re

target_column = df['Text']

non_alphanumeric_characters = {}

    # Iterate over the values in the specific column
for value in target_column:
    #print(value)
        
    # Use a regular expression to find non-alphanumeric characters
    ats = []
    ats = re.findall(r"%", value)
    characters = re.findall(r'[^a-zA-Z0-9\s]', value)
    for character in characters:
        if character not in non_alphanumeric_characters:
            non_alphanumeric_characters[character] = value

    if ats != []:
        print(value)

I think the 90 day rule applies to increases over 5%?
100%. She's made herself vulnerable to being sued by a few people.
If only. He’s signing with the red wings 100%.
Deadlift and 3-lift total 100% raw federation
100%,I'm actually impressed
Be careful, or he'll have to use 10% of his power on you!
I’m glad you fixed Minnesota, because there is a 0% chance we will go Red.
As high as ~72% in black families. Truly heartbreaking what the expansion of the welfare state has done to the black family.
Almost the same Win% as [NAME]. Nice.
Fuck that its not my money they're giving him. I want a goalie with a .970 sv% dangnabbit.
He's worth it I think. Dude brings it 110% every game
Just a quick correction: if the next tax bracket is 10% than $10001 would be taxed $500.10
I used to have anxiety when calling people but I no longer have them at least just 20-30% anxiety.
Absolutely. 100%.
All the things you mentioned are basically aligned perfectly in 99% of cases.
I switched to Ally and love it.

In [ ]:
print("Non-alphanumeric characters in the text:")
for ch, sent in non_alphanumeric_characters.items():
    print(ch)
    print(sent)

In [63]:
def process_slash(text):
    text = re.sub(r'(?<=\s)[rR]/', '', text)
    text = re.sub(r"^(?:r/|R/)", '', text)
    text = re.sub(r'(\w+)\s*/\s*(\w+)', r'\1 or \2', text)
    return text

def process_more(text):
    text = re.sub(r'\+(?:\d+)', ' more ', text)
    return text

def process_and(text):
    text = re.sub(r'\s*&\s*', ' and ', text)
    return text

def specific_case(text):
    text = re.sub(r'\sbi\s', ' bisexual ', text)
    text = re.sub(r'9-1-1', '911', text)
    text = re.sub(r'0-0-0-0-0-10-0-0-01-0-01-0-0-10-0-0', '', text)
    text = re.sub(r'(\d+)ish', r'\1', text)
    text = re.sub(r't@gged', 'tagged', text)
    text = re.sub(r'@.@', '', text)
    text = re.sub(r'🐇', 'rabbit', text)
    text = re.sub(r'I\'m', 'I am', text)
    text = re.sub(r'≠', ' does not equal ', text)
    text = re.sub(r'you[´\']re', ' you are ', text)
    text = re.sub(r"🤰", ' emoji ', text)
    patterns = [r'\(fæ-shē\)', r'\/ˈsatʌɪə\/', r'\/ˈteCHē\/', r'\/ˈbɪɡət\/']
    for pattern in patterns:
        text = re.sub(pattern, '', text)
    text = re.sub(r"xıs :ɹǝʍsuɐ", ' answer: six ', text)
    text = re.sub(r"\[NAME\]", ' user ', text)
    text = re.sub(r"pathetic-ness", ' patheticness ', text)
    result = result.replace("ó", "o")
    result = result.replace("ñ", "n")
    result = result.replace("é", "e")
    result = result.replace("ň", "n")
    result = result.replace("Я", "r")
    result = result.replace("ø", "o")
    result = result.replace("á", "a")
    result = result.replace("ī", "i")
    result = result.replace("ï", "i")
    result = result.replace("🅱", "b")
    result = result.replace("ò", "o")
    result = result.replace("ā", "a")
    result = result.replace("ú", "u")
    result = result.replace("è", "e")
    result = result.replace("Á", "A")
    result = result.replace("ç", "c")
    
    return text

def process_money(text):
    text = re.sub(r"(\d+)\s*\€", r' \1 euros ', text)
    text = re.sub(r"(\d+)\s*\$", r' \1 dollars ', text)
    return text

def process_hyphen_decimal_places(text):
    text = re.sub(r'(\d+(?:\.\d*)?)\s*-\s*(\d+(?:\.\d*)?)', r'\1 to \2', text)
    return text

def process_at(text):
    text = re.sub(r'@(\w+)', r'\1', text)
    text = re.sub(r'@', ' at ', text)
    return text



In [65]:
import re

# Example strings
strings = ["/rThis is a test", "/RAnother test", "This is not /r a test"]

# Regular expression pattern
pattern = r"^(?:/r|/R)"

# Check if the pattern matches at the start of each string
for string in strings:
    if re.search(pattern, string):
        print(f"Matched: {string}")
    else:
        print(f"Not Matched: {string}")


Matched: /rThis is a test
Matched: /RAnother test
Not Matched: This is not /r a test


In [70]:
import re

# Input text containing words separated by forward slashes
text = "OpenAI is / an advanced / AI model/method."

# Using the regex to replace "/" with "to" between words
text = re.sub(r'(\w+)\s*/\s*(\w+)', r'\1 or \2', text)

# Display the modified text
print(text)


OpenAI is or an advanced or AI model or method.


In [97]:
import re

text = "The range is 12 - 67 and another range: 0 - 1.5."

# Use re.sub() to replace hyphens with "to" for numbers with optional decimal places
text = re.sub(r'(\d+(?:\.\d*)?)\s*-\s*(\d+(?:\.\d*)?)', r'\1 to \2', text)

print(text)


The range is 12 to 67 and another range: 0 to 1.5.


In [104]:
import re

text = "This is 123ish, and that's 45ish and 45678ish."

# Use re.sub() to replace 'ish' with an empty string
text = re.sub(r'(\d+)ish', r'\1', text)

print(text)


This is 123, and that's 45 and 45678.


In [106]:
import re

text = "I paid 50€ for the item, and  100   € for the other one."

# Use re.sub() to replace Euro amounts with ' euros '
text = re.sub(r"(\d+)\s*\€", r' \1 euros ', text)

print(text)


I paid  50 euros  for the item, and   100 euros  for the other one.


In [108]:
import re

text = '@love is great'

# Use re.sub() to replace '@word' with what comes after '@'
text = re.sub(r'@(\w+)', r'\1', text)

print(text)


love is great


In [131]:
import re

text = "I have +3 apples, and John has +5 oranges."

# Use re.sub() to replace numbers with '+' with ' more '
text = re.sub(r'(?<=\d)\+', ' more ', text)

print(text)


I have +3 apples, and John has +5 oranges.


In [132]:
import re

text = "I have +3 apples, and John has 4+5 oranges."

# Replace '+' after a digit with ' more '
result = re.sub(r'(?<=\d)\+', ' more ', text)

# Replace '+' after a space with ' plus '
result = re.sub(r'(?<=\s)\+', ' plus ', result)

print(result)


I have  plus 3 apples, and John has 4 more 5 oranges.
